### Fetch Rewards Coding Exercise - Data Analyst - Part III
  - Xiangyu Huang 
  - Sep 28th, 2022

In [1]:
import pandas as pd
import json
from datetime import datetime

In [2]:
receipts = pd.read_json('receipts.json',lines = True)
users = pd.read_json('users.json', lines = True)
brands = pd.read_json('brands.json', lines = True)

In [3]:
receipts.head(3)

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b


In [4]:
len(receipts)

1119

In [5]:
brands.head(3)

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176


In [6]:
len(brands)

1167

In [7]:
users.head(3)

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [8]:
len(users)

495

- **Some quick and easy clean up by implementing lambda function to get rid of the object identifier(oid) symbols for both user's _id and brands _id**


In [9]:
receipts['receipt_id'] = receipts['_id'].apply(lambda row : row['$oid'])
users['user_id'] = users['_id'].apply(lambda row : row['$oid'])
brands['brand_id'] = brands['_id'].apply(lambda row : row['$oid'])

####  Data Quality Examine for the Receipt Table:
     - Check if there is any null values for the key columns such as receipts_id and userId.
     - Check if there is any duplicate receipts id.
     - Check if all the userId are in the users table, and identify the receipts when its userId doens't have match for the User Table's user_id.

In [10]:
receipts['receipt_id'].isnull().sum()  ## no null value for receipts id

0

In [11]:
receipts['userId'].isnull().sum()  ## no null value for userID

0

In [12]:
receipts.groupby('receipt_id').size().sort_values(ascending = False)  ## all the receipt_id are unique.

receipt_id
5f9c74f70a7214ad07000037    1
601c27870a720f05f400029e    1
601c540f0a720f05f40002be    1
601c40230a720f05f40002b5    1
601c2d630a7214ad280002a7    1
                           ..
60088a460a720f05fa0000d5    1
60088a490a720f05fa0000d6    1
60088a4a0a7214ad890000c5    1
60088a4d0a7214ad890000c7    1
603d760e0a720fde1000048e    1
Length: 1119, dtype: int64

In [13]:
receipts.loc[receipts['userId'].isin(users['user_id']) == False, ['receipt_id', 'userId']] 
## where receipt table's userID doesn't match with the user table's user_id.

,receipt_id,userId
13,5f9c74f70a7214ad07000037,5f9c74f7c88c1415cbddb839
15,5ff1e1e90a7214ada1000569,5ff1e1e9b6a9d73a3a9f10f6
16,5ff1e1df0a7214ada1000564,5ff1e1dfcfcf6c399c274ab3
20,5f9c74f90a7214ad07000038,5f9c74e3f1937815bd2c1d73
21,5ff1e1960a720f0523000567,5ff1e196cfcf6c399c274a38
...,...,...
955,602538740a7214d8e9000246,60253861efa6017a44dc6b50
956,602538920a720f05a8000216,60253891b54593795bf69242
966,602538960a720f05a800021b,60253891b54593795bf69242
985,60268c7e0a7214d8e9000309,60268c7bb545931ac63683af


####  Data Quality Examine for the Users dataset
     - Check if there is any missing values for primary key user_id
     - Check if there is any duplicate user_id and identify them


In [14]:
users['user_id'].isnull().sum()  ## no null value for key user_id

0

In [15]:
users['user_id'].duplicated().sum() ## duplicate results

283

In [16]:
user_cnt = users.groupby('user_id').size().sort_values(ascending = False)
user_cnt

user_id
54943462e4b07e684157a532    20
5fc961c3b8cfca11a077dd33    20
5fa41775898c7a11a6bcef3e    18
59c124bae4b0299e55b0f330    18
5ff5d15aeb7c7d12096d91a2    18
                            ..
5ff3711e62fde912123a620e     1
60074883325c8a1794622a12     1
60074b996e64691717e8f11a     1
60083a1e325c8a17946255de     1
60268c7aefa6011bb1510786     1
Length: 212, dtype: int64

In [17]:
users.loc[users['user_id'].duplicated() == True, 'user_id']

1      5ff1e194b6a9d73a3a9f1052
2      5ff1e194b6a9d73a3a9f1052
4      5ff1e194b6a9d73a3a9f1052
5      5ff1e194b6a9d73a3a9f1052
8      5ff1e194b6a9d73a3a9f1052
                 ...           
490    54943462e4b07e684157a532
491    54943462e4b07e684157a532
492    54943462e4b07e684157a532
493    54943462e4b07e684157a532
494    54943462e4b07e684157a532
Name: user_id, Length: 283, dtype: object

In [18]:
for i in range(user_cnt.size):
    if user_cnt[i] == 1:
        endidx = i
        break;
duplicate_user_cnt = user_cnt[0:endidx]
duplicate_user_cnt   
## Identify 70 duplicated user_id with their duplicate counts

user_id
54943462e4b07e684157a532    20
5fc961c3b8cfca11a077dd33    20
5fa41775898c7a11a6bcef3e    18
59c124bae4b0299e55b0f330    18
5ff5d15aeb7c7d12096d91a2    18
                            ..
5ff873ddb3348b11c9337733     2
60229990b57b8a12187fe9e0     2
60074b49325c8a1794623876     2
6002541ae257124ec6b99a3a     2
5ff8da28b3348b11c9337ac6     2
Length: 70, dtype: int64

####  Data Quality Examine for the Brands dataset
     - Check if there is any missing values for key fields like barcode and brand_id
     - Check if there is any duplicate brand_id, barcode and identify them

In [19]:
brands['brand_id'].isnull().sum()  ## no null value for brand_id

0

In [20]:
brands['barcode'].isnull().sum()  ## no null value for barcode

0

In [21]:
brand_cnt = brands.groupby('brand_id').size().sort_values(ascending = False)
brand_cnt
## no duplicate brand_id

brand_id
5332f5ebe4b03c9a25efd0a7    1
5f4a7a5dbe37ce2d95e65ca3    1
5f493e72be37ce64d0ae36c3    1
5f493e72be37ce64d0ae36c1    1
5f493dacbe37ce64d0ae36bf    1
                           ..
5a5d26c7e4b06ba572cf24a5    1
5a5d2695e4b0db471c2d042d    1
5a5d263ee4b06ba572cf24a4    1
5a5d25b6e4b0db471c2d042c    1
6026d757be37ce6369301468    1
Length: 1167, dtype: int64

In [22]:
barcode_cnt = brands.groupby('barcode').size().sort_values(ascending = False)
barcode_cnt
## There are some duplicate barcodes

barcode
511111504139    2
511111305125    2
511111704140    2
511111605058    2
511111004790    2
               ..
511111303893    1
511111303732    1
511111303664    1
511111303503    1
511111919803    1
Length: 1160, dtype: int64

In [23]:
brands.loc[brands['barcode'].duplicated() == True, 'barcode']
## Identify 7 duplicate barcodes

299     511111504139
412     511111504788
536     511111204923
651     511111305125
1012    511111605058
1015    511111704140
1071    511111004790
Name: barcode, dtype: int64

 - **Summary of Findings**
     - There are many duplicates for the user_id in table Users and a few duplicates of barcode for table Brands.
         - The duplicate results may lead to inaccurate report in future query or analysis.
     - Also, there are 148 userID appears in table Receipts but not appear in Users's table. 
         - In this case, when we query between the Users table and Receipts table, we may lost some information. 
 
 - **Future Improvement**
     - Clearly, many datetime data was missing, in future, Maybe we should check if we are missing date information from on some dates or during some periods.
     - Also, may be we should check if all the barcode in table Receipts presents in table Brands.